In [1]:
import numpy as np


SAMPLE_SIZE = 10_000
np.random.seed(2025)  # Return same sample every time

random_noise = np.random.normal(0, 1, SAMPLE_SIZE)

x1 = np.random.uniform(0, 1, SAMPLE_SIZE)
x2 = np.random.uniform(0, 1, SAMPLE_SIZE)
y = x1*x2 + x1**2 + np.exp(np.sin(x2)) + random_noise
y = y.reshape((SAMPLE_SIZE, -1))

In [2]:
FEATURES = 2
HIDDEN_LAYERS_SIZES = [2]
OUPUTS = 1

xh_weights = np.ones(shape=(HIDDEN_LAYERS_SIZES[0], FEATURES)).T
xh_biases = np.zeros(shape=(HIDDEN_LAYERS_SIZES[0]))

hy_weights = np.ones(shape=(OUPUTS, HIDDEN_LAYERS_SIZES[-1])).T
hy_biases = np.zeros(shape=(OUPUTS))

In [3]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def feed_forward(x, xh_weights, xh_biases, hy_weights, hy_biases):
    h_star = np.matmul(x, xh_weights) + xh_biases
    h = sigmoid(h_star)

    y_star = np.matmul(h, hy_weights) + hy_biases
    y = sigmoid(y_star)

    return y

X = np.array([x1, x2]).T.reshape((SAMPLE_SIZE, FEATURES))
y_hat = feed_forward(X, xh_weights, xh_biases, hy_weights, hy_biases)

def MSE(y, y_hat):
    return np.mean(np.array(y - y_hat)**2)

MSE(y, y_hat)

np.float64(3.4072872856326546)

In [4]:
print(f"""
Mean: {np.mean(y):.3f}
Min: {np.min(y):.3f}
Max: {np.max(y):.3f}
""".strip())

Mean: 2.209
Min: -2.003
Max: 7.503


In [5]:
def feed_forward(x, xh_weights, xh_biases, hy_weights, hy_biases):
    h_star = np.matmul(x, xh_weights) + xh_biases
    h = sigmoid(h_star)

    y_star = np.matmul(h, hy_weights) + hy_biases
    y = y_star

    return y

y_hat = feed_forward(X, xh_weights, xh_biases, hy_weights, hy_biases)
MSE(y, y_hat)

np.float64(1.8697041275097268)

In [6]:
def backpropagation(X, y, xh_weights, xh_biases, hy_weights, hy_biases, lr):
    # Inference
    h_star = np.array(np.matmul(X, xh_weights) + xh_biases)
    h = sigmoid(h_star)

    y_star = np.array(np.matmul(h, hy_weights) + hy_biases)
    y_hat = y_star

    # Derivatives
    dy_yhat = 2 / SAMPLE_SIZE * (y - y_hat)
    dystar_hstar = h * (1 - h)
    dystar_hstar = np.array(dystar_hstar * hy_weights.reshape(1, -1))
    dystar_hstar = dy_yhat * dystar_hstar

    # Update weights and biases
    hy_biases += lr * np.sum(dy_yhat, axis=0)
    hy_weights += lr * np.sum(dy_yhat * h, axis=0).reshape(-1, 1)

    xh_biases += lr * np.sum(dystar_hstar, axis=0)
    xh_weights += lr * np.matmul(X.T, dystar_hstar)

    return MSE(y, y_hat)

In [7]:
EPOCHS = 100
losses = []
for i in range(EPOCHS):
    ith_loss = backpropagation(X, y, xh_weights, xh_biases, hy_weights, hy_biases, 0.01)
    losses.append(ith_loss)

In [8]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(list(range(EPOCHS)), losses)
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.set(xlabel="Epoch", ylabel="MSE")

plt.savefig(f"assets/images/training({HIDDEN_LAYERS_SIZES[0]}).png", transparent=True)
plt.close()

In [9]:
FEATURES = 2
HIDDEN_LAYERS_SIZES = [2, 4, 8, 16]
OUPUTS = 1
EPOCHS = 100
plot_data = {"epoch": list(range(EPOCHS))}

for size in HIDDEN_LAYERS_SIZES:
    xh_weights = np.ones(shape=(size, FEATURES)).T
    xh_biases = np.zeros(shape=(size))

    hy_weights = np.ones(shape=(OUPUTS, size)).T
    hy_biases = np.zeros(shape=(OUPUTS))

    plot_data[size] = []
    for i in range(EPOCHS):
        ith_loss = backpropagation(X, y, xh_weights, xh_biases, hy_weights, hy_biases, 0.01)
        plot_data[size].append(ith_loss)

In [10]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))

for size in HIDDEN_LAYERS_SIZES:
    ax.plot(plot_data["epoch"], np.log(plot_data[size]), label = f"{size} Neurons")

ax.legend()
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.set(xlabel="Epoch", ylabel="MSE")

plt.savefig(f"assets/images/training.png", transparent=True)
plt.close()